In [1]:
import subprocess
import csv

def split_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks



In [2]:

def call_ollama(prompt):
    command = ['ollama', 'generate', '-m', 'llama-3.1']
    result = subprocess.run(command, input=prompt, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return result.stdout

def extract_code_disease_pairs(output):
    pairs = []
    lines = output.strip().split('\n')
    for line in lines:
        # Assuming the output is in the format '코드,질병명'
        if ',' in line:
            code, disease = line.split(',', 1)
            code = code.strip()
            disease = disease.strip()
            pairs.append((code, disease))
    return pairs

In [7]:
def main():
    # Read the text file
    with open('icd.txt', 'r', encoding='utf-8') as f:
        text = f.read()

    # Split the text into chunks
    chunks = split_text(text)

    all_pairs = set()
    i = 0

    for chunk in chunks:
        prompt = f"다음 텍스트에서 모든 코드와 해당 질병명을 찾아서, '코드,질병명' 형식으로 각 줄에 하나씩 출력하세요.\n\n예:\n1A00, Cholera\n\n텍스트:\n{chunk}"
        print(f"prompt {i} executed")
        output = call_ollama(prompt)
        pairs = extract_code_disease_pairs(output)
        all_pairs.update(pairs)
        i += 1

    # Write to CSV
    with open('output.csv', 'w', encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['코드', '질병명'])
        for code, disease in sorted(all_pairs):
            writer.writerow([code, disease])


In [11]:
def call_ollama(prompt):
    # ollama 명령어와 옵션을 설정합니다.
    command = ['ollama', 'generate', '-m', 'llama-3.1']

    # subprocess를 사용하여 ollama를 호출하고, 프롬프트를 입력으로 전달합니다.
    result = subprocess.run(command, input=prompt, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # ollama의 출력을 반환합니다.
    return result.stdout


In [13]:
print(call_ollama("hello world"))

In [14]:
import re

def extract_matching_lines(input_file, output_file):
    # 코드 패턴 (숫자 4자리와 대문자 알파벳으로 이루어진 코드)
    pattern = r'[0-9A-Z]{4}'

    # 결과를 저장할 리스트
    matching_lines = []

    # 텍스트 파일을 읽기 모드로 열기
    with open(input_file, 'r', encoding='utf-8') as f:
        # 각 줄을 순차적으로 읽기
        lines = f.readlines()

    # 각 줄에 대해 패턴 매칭
    for line in lines:
        # 패턴이 줄에 매칭되면 해당 줄을 결과 리스트에 추가
        if re.search(pattern, line):
            matching_lines.append(line.strip())  # 줄 끝의 공백 제거

    # 결과를 파일에 저장
    with open(output_file, 'w', encoding='utf-8') as out_f:
        for matched_line in matching_lines:
            out_f.write(matched_line + '\n')

    return matching_lines

# 실행 예시
input_file = 'icd.txt'  # 입력 파일명
output_file = 'output.txt'  # 출력 파일명

matching_lines = extract_matching_lines(input_file, output_file)

# 매칭된 줄들을 출력
for line in matching_lines:
    print(line)


© World Health Organization 2022
Suggested citation. International Classification of Diseases, Eleventh Revision (ICD-11). Geneva: World Health Organization; 2022. License: CC BY-ND 3.0 IGO.
The use of ICD-11 in software products is subject to the following license:  https://icd.who.int/en/docs/ICD11-license.pdf. Under the terms of this license, users may not:
TABLE OF CONTENTS
CHAPTER 01
Code range starts with 1A00
Exclusions:	Infection arising from device, implant or graft, not elsewhere classified (NE83.1)
Coded Elsewhere:	Infections of the fetus or newborn (KA60-KA6Z)
Human prion diseases (8E00-8E0Z)
Pneumonia (CA40)
Gastroenteritis or colitis of infectious origin (1A00‑1A40.Z)
Bacterial intestinal infections (1A00‑1A0Z)
Exclusions:	Bacterial foodborne intoxications (1A10‑1A1Z)
Coded Elsewhere:	Abdominal actinomycosis (1C10.1)
Listerial gastroenteritis (1C1A.Y)
1A00  	Cholera
Cholera is a potentially epidemic and life-threatening infection of the intestine, characterised by extreme

In [15]:
import re

def split_lines_based_on_pattern(input_file, output_file1, output_file2):
    # 코드 패턴 (숫자 또는 대문자로 시작하는 4자리 코드)
    pattern = r'^[0-9A-Z]{4}'

    # 두 개의 출력 파일을 쓰기 모드로 열기
    with open(output_file1, 'w', encoding='utf-8') as out_f1, \
         open(output_file2, 'w', encoding='utf-8') as out_f2:
        
        # 입력 파일 읽기
        with open(input_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        # 각 줄에 대해 패턴을 확인하고 적절한 파일에 저장
        for line in lines:
            if re.match(pattern, line):
                out_f1.write(line)  # 패턴에 맞는 줄은 output1.txt에 저장
            else:
                out_f2.write(line)  # 패턴에 맞지 않는 줄은 output2.txt에 저장

# 실행 예시
input_file = 'output.txt'  # 입력 파일명
output_file1 = 'output1.txt'  # 코드로 시작하는 줄 저장 파일
output_file2 = 'output2.txt'  # 코드로 시작하지 않는 줄 저장 파일

split_lines_based_on_pattern(input_file, output_file1, output_file2)


In [18]:
import re

def convert_to_csv(input_file, output_file):
    # 입력 파일을 읽기 모드로 열기
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # 결과를 저장할 리스트
    converted_lines = []

    # 각 줄에 대해 처리
    for line in lines:
        # 공백 (탭 포함)을 쉼표로 바꾸기 (첫 번째 공백만 변경)
        converted_line = re.sub(r'\s+', ',', line, count=1)
        converted_lines.append(converted_line.strip())  # 양 끝 공백 제거 후 저장

    # 변환된 결과를 CSV 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as out_f:
        for converted_line in converted_lines:
            out_f.write(converted_line + '\n')

# 실행 예시
input_file = 'output1.txt'  # 입력 파일명
output_file = 'output.csv'  # CSV 파일로 저장할 파일명

convert_to_csv(input_file, output_file)


In [19]:
import csv

def process_csv_lines(input_file, output_file):
    # 입력 파일을 읽기 모드로 열기
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # CSV 파일로 저장 (따옴표로 쉼표 포함된 필드 처리)
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)

        for line in lines:
            # 각 줄을 첫 번째 쉼표 기준으로 코드와 설명을 나눕니다.
            # 설명 부분에 쉼표가 포함될 수 있으므로 이를 따옴표로 감싸줍니다.
            parts = line.strip().split(',', 1)  # 첫 번째 쉼표만 기준으로 분리
            writer.writerow(parts)

# 실행 예시
input_file = 'input.txt'  # 입력 파일명
output_file = 'output111111.csv'  # CSV 파일로 저장할 파일명

process_csv_lines(input_file, output_file)
